# Assignment Week 4 - The Battle of Neighborhoods

## Table of Content

* [Introduction](###-1.-Introduction)
* [Data](###-2.-Data)

 


### 1. Introduction 

Melbourne is a large city of the Victoria state in Australia with a population of more than 5 million. Due to the diversity of people living in Australia and especially in Melbourne, there are various types of cusines available. There is a high competition between restaurants in the suburbs with a high concentration of food stores and restaurants. Recently, a client who is going to open a restaurant in Melbourne came to us to ask about the best place for openning a new restaurant.  

**Problem description**\
This project therefore aims to find the best place for a resturant in Melbourne, Australia. Our client has asked us to find the best suburb in which there are fewer competitor. Since the restaurant is going to be open until late night the client has concerns about the crime in that suburb. As there is a trade-off between the crime rate and the suitablity of a suburb for a restaurant, at the end of this project the followings are deleivered to provide an insight into different option and their associated costs for our client:

1. A list of suburbs with small number of competitors.
2. A list of suburbs considering the competetion and crime rate.
3. Our recommended list of top 5 suburbs for openning the restaurant.

### 2. Data

For this project, we need to first obtain the geographical data of all suburbs along with their postcode. This requires the following packages to be imported.

In [1]:
import pandas as pd
import numpy as np
%pip install lxml html5lib beautifulsoup4
%pip install requests
import requests
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


---- 
### Geographical data
The postcode data of various suburbs of Melbourne along with their latitude and longitude are available in the following `url`. We create a BeautifulSoup object to webscrapping the data.

In [2]:
url="https://www.matthewproctor.com/full_australian_postcodes_vic"
page=requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

----
Now, let's extract the table data from that and save them in a dataframe `df`.

In [3]:
table1=soup.find("table",attrs={"class":"table"})

row=table1.find_all('tr')
cols=[v.text.replace('\n','') for v in row[0].find_all('td')]

df=pd.DataFrame(columns=cols)
for i in range(1,len(row)):
    tds=row[i].find_all('td')
    vals=[td.text.replace('\n','') for td in tds]
    df=df.append(pd.Series(vals,index=cols),ignore_index=True)

df.head()

,ID,Postcode,Locality,State,Longitude,Latitude,Category,Type,SA3,SA3 Name,SA4,SA4 Name,Region,Status
0,4746,3000,MELBOURNE,VIC,144.956776,-37.817403,Delivery Area,Delivery Area,20604,Melbourne City,206,Melbourne - Inner,R1,Updated 6-Feb-2020
1,4747,3001,MELBOURNE,VIC,144.76592,-38.365017,Post Office Boxes,Post Office Boxes,20605,Port Phillip,206,Melbourne - Inner,R1,Updated 25-Mar-2020 SA3
2,4748,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,Delivery Area,Delivery Area,20604,Melbourne City,206,Melbourne - Inner,R1,Updated 6-Feb-2020
3,4749,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,Delivery Area,Delivery Area,20604,Melbourne City,206,Melbourne - Inner,R1,Updated 6-Feb-2020
4,4750,3004,MELBOURNE,VIC,144.970161,-37.844246,Delivery Area,Delivery Area,20605,Port Phillip,206,Melbourne - Inner,R1,Updated 6-Feb-2020


----
Let's drop those columns that we don't need for this project.

In [4]:
df.drop(['ID','State','Category','Type','SA3','SA4','SA4 Name', 'Region','Status'],axis=1,inplace=True)
df.rename(columns={'Locality':'Suburb','SA3 Name':'Neighborhood'},inplace=True)

In [5]:
df['Suburb']=df['Suburb'].str.replace(" ","_")
df['Suburb']=df['Suburb'].str.pad(10, side ='right', fillchar =' ')
dfnew=df[df["Longitude"] != ""]
dfnew=dfnew[dfnew['Longitude']!='0']
dfnew[['Longitude','Latitude']]=dfnew[['Longitude','Latitude']].astype('float')
df_grouped=dfnew.groupby(['Postcode']).agg({'Longitude':'mean', 'Latitude':'mean','Suburb':'sum'})

In [6]:

df_grouped.reset_index(inplace=True)
#df_grouped['Suburb']=df_grouped['Suburb'].str.replace(" ",",")
df_grouped.head()

,Postcode,Longitude,Latitude,Suburb
0,3000,144.956776,-37.817403,MELBOURNE
1,3001,144.765920,-38.365017,MELBOURNE
2,3002,144.982207,-37.818517,EAST_MELBOURNE
3,3003,144.949592,-37.810871,WEST_MELBOURNE
4,3004,144.970161,-37.844246,MELBOURNE ST_KILDA_ROAD_CENTRAL


----
### Crime data
Now, let's collect the data for the crimes in different suburbs. This data is publicly available at the "https://www.crimestatistics.vic.gov.au/crime-statistics/latest-crime-data/download-data". I already downloaded the .csv file and saved the data as a new dataframe.

In [8]:
df_crime=pd.read_csv("C:/Users/mnaemi/Google Drive/Data Science/Capstone/crime.csv")
df_crime.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2019,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1
1,2019,December,Alpine,3691,Dederang,E Justice procedures offences,E10 Justice procedures,E14 Pervert the course of justice or commit pe...,1
2,2019,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1
3,2019,December,Alpine,3691,Dederang,D Public order and security offences,D20 Disorderly and offensive conduct,D22 Drunk and disorderly in public,2
4,2019,December,Alpine,3691,Dederang,B Property and deception offences,B20 Property damage,B21 Criminal damage,1


Let's drop unnecessary columns.

In [9]:
df_crime.drop(['Year','Year ending','Local Government Area','Suburb/Town Name','Offence Subdivision','Offence Subgroup'],axis=1,inplace=True)
df_crime.rename(columns={'Suburb/Town Name':'Suburb','Offence Division':'Type'},inplace=True)
df_crime.head()



,Postcode,Type,Incidents Recorded
0,3691,B Property and deception offences,1
1,3691,E Justice procedures offences,1
2,3691,A Crimes against the person,1
3,3691,D Public order and security offences,2
4,3691,B Property and deception offences,1


----
Now, we group the data based on the postcode to get the total number of crimes in each region.

In [10]:
df_crime['Incidents Recorded']=df_crime['Incidents Recorded'].astype('string')
df_crime['Incidents Recorded']=df_crime['Incidents Recorded'].str.replace(',','')
df_crime[['Incidents Recorded','Postcode']]=df_crime[['Incidents Recorded','Postcode']].astype('int64')

df_crime=df_crime.groupby(['Postcode']).sum()

In [11]:
df_crime

,Incidents Recorded
Postcode,
3000,148273
3002,7572
3003,6812
3004,7453
3005,671
...,...
3990,50
3991,283
3992,288


----
Now, let's merge two dataframe using their postcode.

In [12]:
df_grouped['Postcode']=df_grouped['Postcode'].astype('int64')
df_main=df_grouped.merge(df_crime,on='Postcode')
df_main1=df_main[['Postcode','Suburb','Longitude','Latitude','Incidents Recorded']]

In [14]:
df_main1.head()

,Postcode,Suburb,Longitude,Latitude,Incidents Recorded
0,3000,MELBOURNE,144.956776,-37.817403,148273
1,3002,EAST_MELBOURNE,144.982207,-37.818517,7572
2,3003,WEST_MELBOURNE,144.949592,-37.810871,6812
3,3004,MELBOURNE ST_KILDA_ROAD_CENTRAL,144.970161,-37.844246,7453
4,3005,WORLD_TRADE_CENTRE,144.950858,-37.824608,671


----
### Foursquare data

Now, we use the foursquare to get the information about the restaurants in Melboune suburbs. First, we need to use our client id and secret for calling Foursquare API.

In [15]:
CLIENT_ID = 'VHKUTYEQ1M5PALROLDQCNEELUV5FUXPLNJAOPO0MWFCCOCTU' # your Foursquare ID
CLIENT_SECRET = 'W0YKRNXEMWDR5WPEMZHP53XRD24ZM3FF4SICPK2KSR25BKIH' # your Foursquare Secret3
cat_id='4d4b7105d754a06374d81259' # food venues id
neighborhood_latitude=-37.817403
neighborhood_longitude=144.956776
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


----
The following function get the venues in a suburb and return them.

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            cat_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

----
Let's explore each postcode to find the venues in each of them and make a dataframe for that.

In [23]:
df_100=df_main.head(100)
Melb_venues = getNearbyVenues(names=df_100['Suburb'],
                                   latitudes=df_main1['Latitude'],
                                   longitudes=df_main1['Longitude']
                                  )

In [24]:
Melb_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MELBOURNE,-37.817403,144.956776,Brim CC,-37.817764,144.954732,Japanese Restaurant
1,MELBOURNE,-37.817403,144.956776,Royal Stacks,-37.817867,144.958489,Burger Joint
2,MELBOURNE,-37.817403,144.956776,The Grain Store,-37.819562,144.956819,Café
3,MELBOURNE,-37.817403,144.956776,Kinfolk,-37.816996,144.953836,Café
4,MELBOURNE,-37.817403,144.956776,Mason Dixon American Sandwich Bar,-37.817750,144.957980,Sandwich Place
